<a href="https://colab.research.google.com/github/IamArmanNikkhah/ServerClientApp/blob/main/Server_Client_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import socket
import threading

class Server:
  def __init__(self, port = 5050):
    self.PORT   = port
    self.SERVER = socket.gethostbyname(socket.gethostname())
    self.ADDR   = (self.SERVER, self.PORT)  #used for socket.bind
    self.Socket = None
    self.FORMAT = 'utf-8'
    self.DISCONNECT_MESSAGE = "!DISCONNECT"
    self.HEADER = 64              #the first massage that every client send is not the actual massage. instead it sends a 64 byte massage contains the lenght of
                                  # next massage that is the actual massage whick we want to make it upper case.
    
   
  def handle_client(self, conn, addr):
    """
    This function handles a client connection.

    Parameters:
        self (class): The class itself, in this case Server.
        conn (socket): The connection socket.
        addr (tuple): The address tuple.

    Returns:
        None 
    """
    
    print(f"[NEW CONNECTION] {addr} connected.")

    connected = True
    while connected:
        msg_length = conn.recv(self.HEADER).decode(self.FORMAT)
        if msg_length:
            msg_length = int(msg_length)
            msg = conn.recv(msg_length).decode(self.FORMAT)
            if msg == self.DISCONNECT_MESSAGE:
                connected = False
                print("[CONNECTION CLOSED]")
            else:
              print(f"[{addr}] {msg}")
              modified = msg.upper()
              conn.send(modified.encode(self.FORMAT))

    conn.close()
  
  def setup_server(self):
    pass
  
  def start_server(self):
    """This function starts the server.
    It creates a socket object and binds it to the host and port.
    It then listens for incoming connections.
    When a connection is established, it starts a new thread for the client.
    It then goes back to listening for new connections."""

    self.setup_server()
    self.Socket.listen()
    print(f"[LISTENING] Server is listening on {self.SERVER}")
    while True:
        conn, addr = self.Socket.accept()
        # starting a new thread for handling the client
        thread = threading.Thread(target=self.handle_client, args=(conn, addr))
        thread.start()
        print(f"[ACTIVE CONNECTIONS] {threading.activeCount() - 1}")


class TCPserver(Server):
  def __init__(self, port = 5050):
    super().__init__(port)

  def setup_server(self):
    """
    This function sets up the server by creating a socket object and binding it to the address given by the ADDR tuple.
    The socket is then returned.
    
    Parameters:
        self: The server object.
        
    Returns:
        self.Socket: The socket object.
    """
    self.Socket = socket(socket.AF_INET, socket.SOCK_STREAM)
    self.Socket.bind(self.ADDR)

class UDPserver(Server):
  def __init__(self, port = 5050):
    super().__init__(port)

  def setup_server(self):
    self.Socket = socket(socket.AF_INET, socket.SOCK_DGRAM)
    self.Socket.bind(self.ADDR)

In [ ]:
class Client:
  def __init__(self, serverip , port = 5050):
    self.PORT     = port
    self.SERVERIP = serverip
    self.ADDR   = (self.SERVERIP, self.PORT)  #used for socket.connect
    self.Socket = None
    self.FORMAT = 'utf-8'
    self.DISCONNECT_MESSAGE = "!DISCONNECT"
    self.HEADER = 64                          #the first massage that every client send is not the actual massage. instead it sends a 64 byte massage contains the lenght of
                                              # next massage that is the actual massage whick we want to make it upper case.


  def setup_client(self):
    pass

  def send(self, msg):
    """
    This function is used to send a message to the server.
    It takes a string as an argument and encodes it to bytes.
    It then sends the length of the massage to the server first.
    It then sends the massage itself.
    """
    message = msg.encode(self.FORMAT)
    msg_length = len(message)
    send_length = str(msg_length).encode(self.FORMAT)
    send_length += b' ' * (self.HEADER - len(send_length))    #pad in order to have 64 byte lenght
    self.Socket.send(send_length)
    self.Socket.send(message)
    return msg_length

  def receive(self, msg_length):
    print(f"[Massage Recived] {self.Socket.recv(msg_length).decode(self.FORMAT)}")


  def start_client(self):
    self.setup_client()
    while True:
      massage = input("[Input Your Massage:] ")
      lenght  = self.send(massage)
      if massage == self.DISCONNECT_MESSAGE:
        print("[CONNECTION CLOSED]")
        break
      self.receive(lenght)


class TCPclient(Client):
  def __init__(self, serverip , port = 5050):
    super().__init__(serverip, port)

  def setup_client(self):
    self.Socket = socket(socket.AF_INET, socket.SOCK_STREAM)
    self.Socket.connect(self.ADDR)

class UDPclient(Client):
  def __init__(self, serverip , port = 5050):
    super().__init__(serverip, port)

  def setup_client(self):
    self.Socket = socket(socket.AF_INET, socket.SOCK_DGRAM)
    self.Socket.connect(self.ADDR)